In [3]:
import geopandas as gpd
import pandas as pd
import shapely
from pathlib import Path
from sqlalchemy import create_engine
from shapely import wkt
from shapely import validation

In [4]:
ibama_path = Path("/home/obscuritenoir/Portfolio/AgroESG-Carbon-Compliance/data/raw/ibama_embargos.csv")
ibama_df = pd.read_csv(
    ibama_path, 
    sep=';', 
    encoding='latin1', 
    on_bad_lines='warn'
)

/tmp/ipykernel_13909/4089828084.py:2: DtypeWarning: Columns (2,4,6,8,21,36,39,46) have mixed types. Specify dtype option on import or set low_memory=False.
  ibama_df = pd.read_csv(


In [11]:
def validar_geometria(valor):
    if pd.isna(valor) or str(valor).strip() == "":
        return "VAZIO"
    try:
        # Tenta transformar o texto em um objeto de geometria real
        forma = wkt.loads(str(valor))
        if forma.is_valid:
            return "CORRETO"
        else:
            return "INVALIDO_GEOMETRICAMENTE" # Ex: linhas que se cruzam
    except Exception:
        return "CORROMPIDO_TEXTO" # Ex: falta um parêntese no WKT

# 3. Aplicar a classificação (Isso cria uma coluna de diagnóstico)
ibama_df['status_limpeza'] = ibama_df['GEOM_AREA_EMBARGADA'].apply(validar_geometria)

# 4. Agora você separa os dados para análise
df_limpo = ibama_df[ibama_df['status_limpeza'] == "CORRETO"]
df_erros = ibama_df[ibama_df['status_limpeza'] != "CORRETO"]

print(f"Sucesso: {len(df_limpo)} linhas")
print(f"Problemas: {len(df_erros)} linhas")

Sucesso: 54722 linhas
Problemas: 57680 linhas
